In [48]:
import pandas as pd
import pickle
import networkx as nx
import numpy as np

from tqdm import tqdm

In [33]:
tmc_attr = pd.read_csv("../data/shapefiles/tmc_attr.csv")
df_tmc_miles = tmc_attr[["id_tmc", "miles_tmc"]]
miles_tmc = (df_tmc_miles.set_index("id_tmc")).to_dict("index")
with open("../data/tmc_miles.pkl", "wb") as f:
    pickle.dump(miles_tmc, f)

In [38]:
def read_shp(path):
    net = nx.read_shp(path, simplify=True)
    return net

def build_tmc_dict(net):
    tmc2edge = { }
    for key in net.edges:
        tmc_id = net.edges[key]['id']
        if tmc_id not in tmc2edge:
            tmc2edge[tmc_id] = [ ]
        tmc2edge[tmc_id].append(key)
    return tmc2edge

def get_upstream_tmc(tmc2edge, tmc_id, net, n=1):
    out = [ ]    
    for (end_node, bgn_node) in tmc2edge[tmc_id]:
        for e in nx.bfs_edges(net, bgn_node ,depth_limit=n):
            out.append(net.edges[e]['id'])
    return np.unique(out)

def get_downstream_tmc(tmc2edge, tmc_id, miles_tmc, net, n=1):
    out = set()    
    for (end_node, bgn_node) in tmc2edge[tmc_id]:
        dist = 5
        for e in nx.bfs_edges(net, end_node ,depth_limit=n, reverse=True):
            # reverve back due to reverse of graph in bfs
            r_e = (e[1],e[0])
            nxt_tmc = net.edges[r_e]['id']
            
            # avoid repeating segments or tmc_id itself
            if nxt_tmc == tmc_id or nxt_tmc in out:
                continue
            else:
                out.add(nxt_tmc)

                # compute the range of nxt_tmc segment
                if nxt_tmc in miles_tmc:
                    d = miles_tmc[nxt_tmc]["miles_tmc"]
                else:
                    d = 0
                    for end, begin in tmc2edge[nxt_tmc]:
                        temp = ((begin[0]-end[0])**2 + (begin[1]-end[1])**2)**(0.5)
                        d += temp
                    d /= len(tmc2edge[tmc_id])
                dist -= d

            if dist <= 0:
                break    
    return list(out)

def get_neighbor_tmc(tmc_id, net, n=5):
    tmc2edge = build_tmc_dict(net)
    up = get_upstream_tmc(tmc2edge, tmc_id, net, n)
    dn = get_downstream_tmc(tmc2edge, tmc_id, net, n)
    return np.concatenate((up, dn))

In [35]:
net = read_shp('../data/shapefiles/tmc_shape_cranberry/cranberry.shp')
tmc2edge = build_tmc_dict(net)

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_61074/540285700.py:2: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  net = nx.read_shp(path, simplify=True)


In [50]:
next_tmc = {}
for tmc in tqdm(tmc2edge):
    next_tmc[tmc] = get_downstream_tmc(tmc2edge, tmc, miles_tmc, net, n=1)

100%|██████████| 315/315 [00:00<00:00, 171562.88it/s]


In [52]:
with open("../data/next_tmc_5_miles.pkl", "wb") as f:
    pickle.dump(next_tmc, f)